# Regression

In [1]:
#r "nuget: Microsoft.ML, 5.0.0"
#r "nuget: Microsoft.ML.FastTree, 5.0.0"
#r "nuget: Microsoft.ML.LightGbm, 5.0.0"
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed Packages Microsoft.ML, 5.0.0 Microsoft.ML.FastTree, 5.0.0 Microsoft.ML.LightGbm, 5.0.0

In [2]:
#r "D:\AI For Developers-The Starting Point\ML.Net\Ml.Net\bin\Debug\net8.0\ML.Net.dll"
using Ml.Net;

In [11]:
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromTextFile<HousingData>(path: "Salary_data.csv", hasHeader: true, separatorChar: ',');

In [4]:
 void DisplayData(DataDebuggerPreview preview)
 {
     Console.WriteLine("\n=== Transformed Data Preview ===\n");
     // Get column names
     var columnNames = preview.Schema.Select(c => c.Name).ToArray();
     // Print headers
     foreach (var col in columnNames)
         Console.Write($"{col,-20}");
     Console.WriteLine();
     Console.WriteLine(new string('-', columnNames.Length * 20));
     // Print rows
     foreach (var row in preview.RowView)
     {
         foreach (var column in row.Values)
         {
             string value = column.Value == null ? "NULL" : column.Value.ToString();
             if (value.Length > 18) value = value.Substring(0, 17) + "..."; // shorten long vector output
             Console.Write($"{value,-20}");
         }
         Console.WriteLine();
     }
     Console.WriteLine("\n============================\n");
 }

In [5]:

var pipeline = mlContext.Transforms.ReplaceMissingValues(new[]
{ 
    new InputOutputColumnPair("Price")},MissingValueReplacingEstimator.ReplacementMode.Mean)
     .Append(mlContext.Transforms.Categorical.OneHotEncoding(new[]{
                new InputOutputColumnPair("City"),
                new InputOutputColumnPair("Locality"),
                new InputOutputColumnPair("Facing"),
                new InputOutputColumnPair("Lift"),
                new InputOutputColumnPair("Furnishing"),
                new InputOutputColumnPair("Society"),
                new InputOutputColumnPair("Garden")
                }))           
            .Append(mlContext.Transforms.Concatenate("Features", "City", "Locality", "Facing", "Lift", "Furnishing", "Society", "Garden", "City", 
            "Bedrooms", "Bathrooms", "SqftLiving", "SqftLot", "Floors", "YearBuilt", "YearRenovated", "Parking", "NearbySchools", "NearbyHospitals", "Balcony"))
            .Append(mlContext.Transforms.NormalizeMinMax("Features"))
            .Append(mlContext.Regression.Trainers.OnlineGradientDescent(labelColumnName: "Price",featureColumnName:"Features"));

In [6]:
// Split and train the model
var split = mlContext.Data.TrainTestSplit(data, testFraction: 0.2);

In [7]:
// Train the model
var model=pipeline.Fit(split.TrainSet);

In [8]:
// Evualuate the model
var prediction=model.Transform(split.TestSet);
var metrics=mlContext.Regression.Evaluate(prediction,labelColumnName:"Price");

In [10]:
Console.WriteLine("===== MODEL METRICS =====");
Console.WriteLine($"RMSE: {metrics.RootMeanSquaredError}");
Console.WriteLine($"MAE: {metrics.MeanAbsoluteError}");
Console.WriteLine($"R²  : {metrics.RSquared}");

===== MODEL METRICS =====
RMSE: 7841282.983544786
MAE: 6576340.147058823
R²  : 0.6803831702051373


In [65]:
// Sample prediction
var engine=mlContext.Model.CreatePredictionEngine<HousingData,Prediction>(model);

In [74]:
 var sample = new HousingData
        {
            City = "Delhi",
            Locality = "Vasant Kunj",
            Bedrooms = 2,
            Bathrooms = 2,
            SqftLiving = 2000,
            SqftLot = 2500,
            Floors = 20,
            YearBuilt = 2000,
            YearRenovated = 2025,
            PropertyType = "Apartment",
            Parking = 0,
            Facing = "North",
            Lift = "No",
            Furnishing = "Furnished",
            NearbySchools = 1,
            NearbyHospitals = 1,
            Balcony = 1,
            Garden = "No"
        };

In [75]:
var predicrtionResult=engine.Predict(sample);
Console.WriteLine($"\nPredicted price for the sample house: {predicrtionResult.Score}");


Predicted price for the sample house: 21037184


In [ ]:
data = pipeline.Fit(data).Transform(data);
var preview = data.Preview(10);
DisplayData(preview);